In [ ]:
# use full screen width

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import os
import re
import time
import copy
import logging
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt


# set logging.INFO for more detailed output, logging.WARNINg to suppress INFO output
logging.basicConfig(level = logging.INFO, force = True)

# function to aggregate weights per country
def aggregate_weights(my_df):
    for _, g in my_df.groupby("Country"):
        # len > 1 -> more than one country
        if len(g) > 1:
            total_weight = 0
            for i in range(len(g)):
                total_weight += my_df.loc[g.index[i], 'Weighting']
            
            # write total weight to first occurence of duplicate
            my_df.loc[g.index[0], 'Weighting'] = total_weight

    # drop all but 1st occurence -> df with total weight per country
    return my_df.drop_duplicates(subset = ['Country'], keep = 'first')


def load_data(data_file):
    df = pd.read_csv(data_file, sep = '\t')
    
    # Keep only columns ISIN and weighting
    df = df[['Country', 'Weighting']]

    return df


# concat dfs with deep copies (orginal dfs intact to etract initial weights)
def concat_dfs(df_list):
    cp_list = copy.deepcopy(df_list)
    for _, df in enumerate(cp_list, 1):
        df['Weighting'] = df['Weighting'].multiply(weights[f'weight_{_}'] * 100)
    
    return pd.concat(cp_list, ignore_index = 'true')  


# load sample data
cwd = os.getcwd()

df_1 = load_data(os.path.join(cwd, 'data/xtrackers_msci-world.csv'))
df_2 = load_data(os.path.join(cwd, 'data/xtrackers_europe-small-cap.csv'))
df_3 = load_data(os.path.join(cwd, 'data/ishares-msci-em-asia_new.csv'))
df_4 = load_data(os.path.join(cwd, 'data/ishares-msci-pacific-exjapan_new.csv'))

# set some initial weights
weights = {
    'weight_1': 0.4,
    'weight_2': 0.4,
    'weight_3': 0.1,
    'weight_4': 0.1,
}

my_df = concat_dfs([df_1, df_2, df_3, df_4])

my_df = aggregate_weights(my_df)

logging.info(my_df.sort_values(by = 'Weighting', ascending = False))

# calculate 'coverage', i.e. percentage of stocks found in all ETFs
cov = my_df['Weighting'].sum()

logging.info(f'Overall: {len(my_df)} countries, coverage {cov:.2f} %')

In [ ]:
# load shapefile (columns ISO_A2, geometry, NAME_DE) with geopandas
map_path = os.path.join(cwd, 'data/map/ne_10m_admin_0_countries.shp')
world = gpd.read_file(map_path)[['ISO_A2', 'geometry', 'NAME_DE']]

# Check if all countries from our data set are found in the map file (caveats e.g. China vs. Volksrepublik China)
if not pd.Series(my_df['Country'].isin(world['NAME_DE']).values, my_df['Country'].values).nunique() == 1:
    logging.warning('Missing country')

# find and drop row corresponding to Antarctica (too big on final map and unnecessary)
logging.debug(world[world['ISO_A2'] == 'AQ'])
world = world.drop(world.index[172])

# merge world with overall weights
world_df = my_df.merge(world, left_on = 'Country', right_on = 'NAME_DE', how = 'right')

logging.info(world_df)

# function to merge df_w with initial weights
def merge_initial_weights(df_list, df_w):
    cp_list = copy.deepcopy(df_list)
    for _, df in enumerate(cp_list, 1):        
        df = aggregate_weights(df)
        
        df.rename(columns = {'Weighting': f'initial_weight_{_}'}, inplace = True)
                
        df_w = df_w.merge(df, on = 'Country', how = 'left')
        
    return df_w

world_df = merge_initial_weights([df_1, df_2, df_3, df_4], world_df)

# print sample for debugging
logging.info(world_df[world_df['ISO_A2'] == 'US'])

In [ ]:
import json

# convert merged df to geodataframe
world_geo_df = gpd.GeoDataFrame(world_df)

# read data as json
world_json = json.loads(world_geo_df.to_json())

# encode data
json_data = json.dumps(world_json)

In [ ]:
from bokeh.io import output_notebook, show, output_file, push_notebook
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, HoverTool, CustomJS, Slider, Dropdown
from bokeh.palettes import brewer, mpl, cividis, Magma256
from bokeh.layouts import column, row

# GeoJSONDataSource containing features for plotting
geosource = GeoJSONDataSource(geojson = json_data)

# colour palette
palette = brewer['YlGnBu'][8]

# reverse colour palette: darker -> higher weight
palette = palette[::-1]

# LinearColorMapper to map numbers into a sequence of colors
color_mapper = LinearColorMapper(palette = palette, nan_color = '#d9d9d9')

# colour bar 
color_bar = ColorBar(color_mapper = color_mapper, label_standoff = 8, width = 500, height = 20, border_line_color = None, location = (0,0), orientation = 'horizontal', title = 'Weight in %')

# tool bar
tools = 'wheel_zoom, pan, reset'

# create sliders to change weights
w1_slider = Slider(start = 0, end = 100, value = weights['weight_1']*100, step = 5, title = 'Xtrackers MSCI World')
w2_slider = Slider(start = 0, end = 100, value = weights['weight_2']*100, step = 5, title = 'Xtrackers MSCI Europe Small Cap')
w3_slider = Slider(start = 0, end = 100, value = weights['weight_3']*100, step = 5, title = 'iShares MSCI EM Asia')
w4_slider = Slider(start = 0, end = 100, value = weights['weight_4']*100, step = 5, title = 'iShares MSCI Pacific ex-Japan')

# custom callback to handle interaction with sliders (update weights)
callback = CustomJS(args = dict(source = geosource, weigth_1 = w1_slider, weigth_2 = w2_slider, weigth_3 = w3_slider, weigth_4 = w4_slider), code="""
   var data = source.data;
   const A = weigth_1.value;
   const B = weigth_2.value;
   const C = weigth_3.value;
   const D = weigth_4.value;
   
   var x = data['Weighting']
   var w = data['initial_weight_1']
   var v = data['initial_weight_2']
   var u = data['initial_weight_3']
   var t = data['initial_weight_4']
   
   if ((A+B+C+D) === 100) {
    for (var i = 0; i < x.length; i++) {
     if (isNaN(w[i])) {
      w[i] = 0;  
     }
     if (isNaN(v[i])) {
      v[i] = 0;  
     } 
     if (isNaN(u[i])) {
      u[i] = 0;  
     }
     if (isNaN(t[i])) {
      t[i] = 0;  
     }
     x[i] = w[i]*A + v[i]*B + u[i]*C + t[i]*D;
     if (x[i] === 0) {
      x[i] = NaN
    }
   }
   }

   
   source.change.emit();
""")

# if (B===0) damit NaN wenn weight 0, also zB nur bei europe small cap US in grau
w1_slider.js_on_change('value', callback)
w2_slider.js_on_change('value', callback)
w3_slider.js_on_change('value', callback)
w4_slider.js_on_change('value', callback)

# create figure object
# active scroll to be active by default
p = figure(plot_height = 750 , plot_width = 1250, toolbar_location = 'right', tools = tools, active_scroll = 'wheel_zoom')#title=''
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# create tooltip to show mouseover with country and according weight
hover = HoverTool(
    tooltips = [
    ('Country', '@NAME_DE'),
    ('Weight', '@Weighting{%.3f} %'),
    ], 
    
    formatters = {
        '@Weighting' : 'printf',
    },
    
    mode = 'mouse'
)

p.add_tools(hover)

# Add patch renderer to figure
p.patches('xs','ys', source = geosource, fill_color = {'field' : 'Weighting', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

# Specify figure layout
p.add_layout(color_bar, 'below')

# dropdown menu change colour
menu = [('Plasma', 'plasma'), ('RdYlGn', 'rdylgn'), ('Cividis', 'cividis'), ('Magma', 'magma'), ('YlGnBu (Default)', 'ylgnbu')]

dropdown = Dropdown(label = 'Colour', button_type = 'warning', menu = menu)

# custom callback to toggle colour changes
callback_dopdown = CustomJS(args = dict(fig = p, col_palette = color_mapper, palette_1 = mpl['Plasma'][11][::-1], palette_2 = brewer['RdYlGn'][11], palette_3 = cividis(50)[::-1], palette_4 = Magma256[::-1], palette_5 = brewer['YlGnBu'][8][::-1]), code="""
    var col = this.item
    switch (col) {
        case "plasma":
            // console.log(col_palette.palette);
            col_palette.palette = palette_1;
            // console.log(col_palette.palette);
            break;
        case "rdylgn":
            col_palette.palette = palette_2;
            break;
        case "cividis":
            col_palette.palette = palette_3;
            break;
        case "magma":
            col_palette.palette = palette_4;
            break;
        case "ylgnbu":
            col_palette.palette = palette_5;
            break;
    
    }
    // console.log(col_palette.palette);
    fig.change.emit();
   
""")
                            
dropdown.js_on_event('menu_item_click', callback_dopdown)

# display figure inline in Jupyter Notebook.
output_notebook()

layout = row(
    p,
    column(w1_slider, w2_slider, w3_slider, w4_slider, dropdown),
)

# display figure, handle not used anymore
h = show(layout, notebook_handle = True)